In [2]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('real_estate.db')

# Query to get all table names
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(query, conn)

print("\nSchema representation for NLP-to-SQL agent:")
schema_representation = ""

for table in tables['name']:
    schema_query = f"PRAGMA table_info({table});"
    schema = pd.read_sql_query(schema_query, conn)
    
    columns = []
    for _, row in schema.iterrows():
        column_name = row['name']
        column_type = row['type']
        columns.append(f"{column_name} <{column_type}>")
    
    schema_representation += f"Table: {table}\n"
    schema_representation += "Columns: " + ", ".join(columns) + "\n\n"

print(schema_representation)

# Close the connection
conn.close()


Schema representation for NLP-to-SQL agent:
Table: listings_buy
Columns: title <TEXT>, price <TEXT>, type <TEXT>, beds <REAL>, studio <TEXT>, baths <INTEGER>, area <TEXT>, location <TEXT>, payment <TEXT>, agent_plan <TEXT>, building_name <TEXT>, street_name <TEXT>, area_name <TEXT>, verified_date <TIMESTAMP>, handover quater <TEXT>, handover year <TEXT>, offplan type <TEXT>, type of sale <TEXT>

Table: listings_rent
Columns: title <TEXT>, price <TEXT>, type <TEXT>, beds <REAL>, studio <TEXT>, baths <INTEGER>, area <TEXT>, location <TEXT>, agent_plan <TEXT>, building_name <TEXT>, street_name <TEXT>, area_name <TEXT>, verified_date <TIMESTAMP>

Table: transactions_buy
Columns: DATE <TIMESTAMP>, LOCATION <TEXT>, Type <TEXT>, Beds <TEXT>, BUILT-UP AREA <TEXT>, FLOOR <TEXT>, BUILT-UP <TEXT>, PLOT <TEXT>, building_name <TEXT>, area_name <TEXT>, PRICE <TEXT>, Info <TEXT>

Table: transactions_rent
Columns: START DATE <TIMESTAMP>, LOCATION <TEXT>, Price <TEXT>, Type <TEXT>, BEDS <TEXT>, AREA (

In [2]:
from langchain_openai import AzureChatOpenAI
from langchain_core.tools import tool
import os
model_name = "gpt-4o"
deployment = "azure-gpt-4o"

api_version = "2024-12-01-preview"
endpoint = os.getenv("AZURE_OPENAI_URL")
subscription_key = os.getenv("AZURE_OPENAI_KEY")

llm = AzureChatOpenAI(
    azure_deployment=deployment,
    azure_endpoint=endpoint,
    api_version=api_version,  
    temperature=0.7,
    api_key=subscription_key
)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_types import AgentType
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate

# Connect to your SQLite database
db = SQLDatabase.from_uri("sqlite:///real_estate.db")

# Define your database schema and field descriptions as a system prompt
schema_description = """
    Database: real_estate.db (SQLite)

    Table: listings_buy (Listings about proprties ready to buy)
    Columns: 
    - title (TEXT): Property title/description
    - price (TEXT): Listed price
    - type (TEXT): Property type (Apartment, Villa, etc.)
    - beds (REAL): Number of bedrooms
    - studio (TEXT): Whether it's a studio apartment
    - baths (INTEGER): Number of bathrooms
    - area (TEXT): Property area in square feet
    - location (TEXT): Full location string
    - payment (TEXT): Payment plan
    - agent_plan (TEXT): Agent information
    - building_name (TEXT): Name of the building
    - street_name (TEXT): Street name
    - area_name (TEXT): Area/neighborhood name
    - verified_date (TIMESTAMP): Date when listing was verified
    - handover quater (TEXT): Quarter of handover
    - handover year (TEXT): Year of handover
    - offplan type (TEXT): Off-plan property type
    - type of sale (TEXT): Initial Sale or Resale

    Table: listings_rent (Listings about proprties ready to rent)
    Columns:
    - title (TEXT): Property title/description
    - price (TEXT): Rental price
    - type (TEXT): Property type (Apartment, Villa, etc.)
    - beds (REAL): Number of bedrooms
    - studio (TEXT): Whether it's a studio apartment
    - baths (INTEGER): Number of bathrooms
    - area (TEXT): Property area in square feet
    - location (TEXT): Full location string
    - agent_plan (TEXT): Agent information
    - building_name (TEXT): Name of the building
    - street_name (TEXT): Street name
    - area_name (TEXT): Area/neighborhood name
    - verified_date (TIMESTAMP): Date when listing was verified

    Table: transactions_buy (Transactions  about properties that were sold)
    Columns:
    - DATE (TIMESTAMP): Transaction date
    - LOCATION (TEXT): Full location string
    - Type (TEXT): Property type
    - Beds (TEXT): Number of bedrooms
    - BUILT-UP AREA (TEXT): Built-up area in square feet
    - FLOOR (TEXT): Floor number
    - BUILT-UP (TEXT): Additional built-up area information
    - PLOT (TEXT): Plot information
    - building_name (TEXT): Name of the building
    - area_name (TEXT): Area/neighborhood name
    - PRICE (TEXT): Transaction price
    - Info (TEXT): Additional information

    Table: transactions_rent Transactions  about properties that were rented)
    Columns:
    - START DATE (TIMESTAMP): Rental start date
    - LOCATION (TEXT): Full location string
    - Price (TEXT): Rental price
    - Type (TEXT): Property type
    - BEDS (TEXT): Number of bedrooms
    - AREA (SQFT) (TEXT): Area in square feet
    - FLOOR (TEXT): Floor number
    - building_name (TEXT): Name of the building
    - area_name (TEXT): Area/neighborhood name
    - DURATION(Months) (TEXT): Rental duration in months
    - Info (TEXT): Additional information (NEW or RENEWAL)
"""

# Initialize the language model with the schema information
#llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

# Create a custom prompt template that includes the schema
system_template = f"""You are an AI assistant that translates natural language to SQL.
{schema_description}

When translating natural language to SQL:
1. Only use tables and columns that exist in the schema
2. Be precise with column names and table relationships
3. Use appropriate SQL syntax for SQLite
4. Return only the SQL query without additional explanation unless asked
"""

system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_message_prompt = HumanMessagePromptTemplate.from_template("{input}")

chat_prompt = ChatPromptTemplate.from_messages([
    system_message_prompt,
    human_message_prompt
])

# Create the SQL toolkit with the custom prompt
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# Create the SQL agent with the custom system prompt
agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    system_message=system_template
)

In [5]:
# Example usage
user_query = "Which area had largest value of sales ?"
result = agent.invoke({"input": user_query})
print(result["output"])



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


listings_buy, listings_rent, transactions_buy, transactions_rent
Invoking: `sql_db_schema` with `{'table_names': 'transactions_buy'}`



CREATE TABLE transactions_buy (
	"DATE" TIMESTAMP, 
	"LOCATION" TEXT, 
	"Type" TEXT, 
	"Beds" TEXT, 
	"BUILT-UP AREA" TEXT, 
	"FLOOR" TEXT, 
	"BUILT-UP" TEXT, 
	"PLOT" TEXT, 
	building_name TEXT, 
	area_name TEXT, 
	"PRICE" TEXT, 
	"Info" TEXT
)

/*
3 rows from transactions_buy table:
DATE	LOCATION	Type	Beds	BUILT-UP AREA	FLOOR	BUILT-UP	PLOT	building_name	area_name	PRICE	Info
2025-04-18 00:00:00	The Torch
Dubai Marina	Apartment	2	1,258	39	None	None	The Torch	Dubai Marina	1,680,000	Vacant at time of sale
2025-04-18 00:00:00	Ocean Heights
Dubai Marina	Apartment	1	1,065	26	None	None	Ocean Heights	Dubai Marina	1,865,582	Rented at time of sale
2025-04-18 00:00:00	Marina Diamond 2
Dubai Marina
Marina Diamonds	Apartment	2	968	10	None	None	Marina Diamond 2	Dubai Marina	1,